In [1]:
import pandas as pd
import time
import random
import numpy as np
from datetime import datetime
import json

import requests
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
from apscheduler.jobstores.mongodb import MongoDBJobStore
from db import client
from db import Information
from db import Url_Recommendations
from db import Reviews_Rating
from db import Pages
from db import Agoda_Activities_Reviews

from bson import ObjectId

In [5]:
def Translated_Data (driver):
    try:
        translated_buttons = driver.find_elements(By.XPATH, '//div[@class="acbf1-box acbf1-fill-inherit acbf1-text-inherit acbf1-mt-16      "]//button')
        for tsl_btn in translated_buttons:
            try:
                tsl_btn.click()
                time.sleep(1)  # đợi load content
            except (ElementClickInterceptedException, StaleElementReferenceException):
                continue  # bỏ qua nếu lỗi click

    except NoSuchElementException:
        pass

In [ ]:
def Scraping_Reviews (driver, url, object_id):
    # authors = driver.find_elements (By.XPATH, '//div[@class="css-1dbjc4n r-13awgt0 r-9aw3ui"]') : Không có
    # Số lần tối đa thử click lại 1 nút
    MAX_RETRIES_PER_BUTTON = 5

    try:
        expand_buttons = driver.find_elements(
            By.XPATH,
            '//div[@class="aaa1e-box aaa1e-fill-inherit aaa1e-text-inherit aaa1e-mt-8      "]'
            '//p[@class="sc-gwsNht Typographystyled__TypographyStyled-sc-1uoovui-0 isoMbC hiXmOF"]//button'
        )

        for btn in expand_buttons:
            while True:
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                    time.sleep(1)  # cho scroll xong
                    btn.click()
                    time.sleep(5)
                    break
                except (ElementClickInterceptedException, StaleElementReferenceException):
                    time.sleep(2)

    except NoSuchElementException:
        # print("Không tìm thấy nút mở rộng nào.")
        pass
        
    titles = driver.find_elements (By.XPATH, '//div[@data-testid="activities-review-content"]//p[@class="sc-gwsNht Typographystyled__TypographyStyled-sc-1uoovui-0 bfilTo UwrgA"]')

    contents = driver.find_elements (By.XPATH, '//div[@class="aaa1e-box aaa1e-fill-inherit aaa1e-text-inherit aaa1e-mt-8      "]')
        
    per_ratings = driver.find_elements (By.XPATH, '//div[@data-testid="review-card-title"]//span[@class="sc-gwsNht Typographystyled__TypographyStyled-sc-1uoovui-0 hQhxAT bmjHjQ"]')
        
    time_reviews = driver.find_elements (By.XPATH, '//div[@class="aaa1e-box aaa1e-fill-inherit aaa1e-text-inherit aaa1e-flex aaa1e-flex-row      "]//div[@class="aaa1e-box aaa1e-fill-inherit aaa1e-text-inherit aaa1e-flex aaa1e-flex-col aaa1e-justify-evenly      "]//span[@class="sc-gwsNht Typographystyled__TypographyStyled-sc-1uoovui-0 isoMbC oWowW"]')

    Sources = driver.find_elements (By.XPATH, '//div[@class="aaa1e-box aaa1e-fill-inherit aaa1e-text-inherit aaa1e-mt-16      "]//span[@class="sc-gwsNht Typographystyled__TypographyStyled-sc-1uoovui-0 isoMbC ktgvnM"]')
        
    # Translated_Titles = driver.find_elements (By.XPATH, '//div[@data-element-name="activities-review-translated"]//p[@class="sc-hoLldG sc-jZhnRx ceImgB iBnGae"]')
    
    # Translated_Contents = driver.find_elements (By.XPATH, '//div[@data-element-name="activities-review-translated"]//p[@class="sc-hoLldG sc-jZhnRx jQsqdf ehUZtX"]')
    
    num_reviews = min(
        len(titles),
        len(contents),
        len(per_ratings),
        len(time_reviews),
        len (Sources)
    )
    
    for i in range(num_reviews):
        review_data = {
            'Activity_Id': object_id,
            "Title": titles[i].text.strip(),
            "Content": contents[i].text.strip(),
            "Per_Rating": per_ratings[i].text.strip(),
            "Time_Review": time_reviews[i].text.strip(),
            "Source": Sources[i].text.strip(),
        }
        metadata = get_metadata      (url)
        if metadata:
            review_data.update (metadata)
            
        try:
            Agoda_Activities_Reviews.insert_one(review_data)
            # print (review_data)
        except Exception as e:
            print(f"❌ Error inserting reviews for {url}: {e}")

In [7]:
# review_blocks = driver.find_elements(By.XPATH, '//div[@data-element-name="activities-review-translated"]')

# for i, block in enumerate(review_blocks):
#     try:
#         title_elem = block.find_element(By.XPATH, './/p[contains(@class, "ceImgB")]')
#         translated_title = title_elem.text.strip()
#     except:
#         translated_title = 'No'

#     try:
#         content_elem = block.find_element(By.XPATH, './/p[contains(@class, "sc-hoLldG sc-jZhnRx jQsqdf ehUZtX")]')
#         translated_content = content_elem.text.strip()
#     except:
#         translated_content = 'No'

#     print(f"🔹 Review {i+1}:")
#     print("  Tiêu đề dịch:", translated_title)
#     print("  Nội dung dịch:", translated_content)
#     print("---")


In [8]:
def wait_for_reviews(driver):
    try:
        WebDriverWait(driver, 12).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='activities-review-distribution-indicator-container']"))
        )
        WebDriverWait(driver, 12).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='activities-review-content']"))
        )
        return True
    except:
        return False

In [ ]:
def scroll_to_bottom(driver, step_delay=0.3, max_scrolls=30):
    """
    Cuộn xuống dưới cùng của trang web một cách tuần tự.
    Thường dùng để buộc các phần tử lazy-load xuất hiện (ví dụ: nút Next, comment, review, ...)
    """
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    for i in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(step_delay)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # print(f"✅ Đã cuộn tới đáy trang sau {i+1} lần.")
            break
        last_height = new_height
    else:
        print("Đã cuộn tối đa mà chưa đến đáy trang.")


In [10]:
def Scraping_Pages(driver, url, object_id):
    wait = WebDriverWait(driver, 15)
    page_count = 1

    # 🌀 Thử tải trang hiện tại tối đa 3 lần
    for retry in range(3):
        if wait_for_reviews(driver):
            break  # ✅ Thành công, thoát vòng lặp retry
        print(f"⏳ Không tìm thấy nội dung, thử lại ({retry + 1}/3)...")
        time.sleep(2)
    else:
        print("⚠️ Không thể tải nội dung sau 3 lần thử. Bỏ qua trang này.")
        return  # ❌ Dừng hẳn việc scrape (hoặc dùng continue nếu muốn bỏ qua trang này và tiếp tục)
    
    while True:
        print(f"🔍 Đang scrape reviews trang thứ {page_count}...")
        # ✅ Nếu đến đây nghĩa là nội dung đã tải → scrape reviews
        time.sleep (2)
        Scraping_Reviews(driver, url, object_id)
        random_delay (2,5)
        
        # 👉 Cố gắng chuyển sang trang kế tiếp
        try:
            # Cuộn xuống cuối trang
            scroll_to_bottom(driver)
            
            next_button = wait.until(EC.presence_of_element_located((
                By.XPATH, "//div[@data-testid='review-pagination-container']//nav[@class='aaa1e-box aaa1e-fill-inherit aaa1e-text-inherit aaa1e-items-center aaa1e-flex      ']//button[@leadingicon='fill.arrow.thin-forward']"
            )))
            if next_button.get_attribute("disabled"):
                print("✅ Đã đến trang cuối cùng (nút Next bị vô hiệu hóa).")
                random_delay()
                break
            else:
                # driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(2)
                page_count += 1
            
        except (NoSuchElementException, TimeoutException):
            print("✅ Không còn trang tiếp theo. Đã scrape xong.")
            random_delay ()
            break
        except StaleElementReferenceException:
            print("⚠️ Nút Next bị stale, thử lại.")
            random_delay ()
            break
        except Exception as e:
            print(f"❌ Lỗi khi xử lý trang {page_count + 1}: {e}")
            random_delay ()
            break

In [72]:
driver.quit ()

In [73]:
driver = Create_WebDriver ()
driver.get ('https://www.agoda.com/vi-vn/activities/detail?activityId=941157&cityId=13170&currency=VND&cid=1922896')
time.sleep (5)
Scraping_Pages (driver, 'a', 'b')

🔍 Đang scrape reviews trang thứ 1...
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #1 do lỗi ElementClickInterceptedException... thử lại
✔ Đã click nút #1
⏳ Chờ để click nút #2 do lỗi ElementClickInterceptedException... thử lại
⏳ Chờ để click nút #2 do lỗi ElementClickInterceptedException... thử lại
✔ Đã click nút #2
⏳ Chờ để click nút #3 do lỗi ElementClickIntercepte

In [42]:
Scraping_Pages (driver, 'a', 'b')

🔍 Đang scrape reviews trang thứ 1...
{'Activity_Id': 'b', 'Title': 'A Great Get-Away', 'Content': 'The trip was excellent! Our tour guide Danny was very informative all throughout our journey out of Saigon. The temple was beautiful, and we even got to see part of a funeral service; the tunnels were...\nHiện nhiều hơn', 'Per_Rating': '5/5 trên Tripadvisor', 'Time_Review': 'Nhận xét vào Tháng 3 2024 trên Tripadvisor', 'Source': 'Do Tripadvisor thu thập', 'SourcePlatform': 'a', 'Date': '2025-06-20', 'Time': '22:22:46'}
{'Activity_Id': 'b', 'Title': 'BK Cu Chi Tunnels and Cao Dai Temple', 'Content': 'The tour guide Luat was good and informative. The pace and the organization of the itenarary were good. The food was above average. The car and driver were good.', 'Per_Rating': '5/5 trên Viator', 'Time_Review': 'Nhận xét vào Tháng 1 2023 trên Viator', 'Source': 'Do Viator thu thập', 'SourcePlatform': 'a', 'Date': '2025-06-20', 'Time': '22:22:46'}
{'Activity_Id': 'b', 'Title': 'Great personali

In [13]:
cursor = Pages.find({}, {"_id": 1, "URL": 1})

items_to_scrape = [{"_id": doc["_id"], "URL": doc["URL"]} for doc in cursor]

items_to_scrape

[{'_id': ObjectId('683dce7105923cb54314fdbe'),
  'URL': 'https://www.agoda.com/vi-vn/activities/detail?activityId=1141667&cityId=13170&currency=VND&cid=1922896'},
 {'_id': ObjectId('683dce7105923cb54314fdbf'),
  'URL': 'https://www.agoda.com/vi-vn/activities/detail?activityId=1045137&cityId=13170&currency=VND&cid=1922896'},
 {'_id': ObjectId('683dce7105923cb54314fdc0'),
  'URL': 'https://www.agoda.com/vi-vn/activities/detail?activityId=825445&cityId=13170&currency=VND&cid=1922896'},
 {'_id': ObjectId('683dce7105923cb54314fdc1'),
  'URL': 'https://www.agoda.com/vi-vn/activities/detail?activityId=1433950&cityId=13170&currency=VND&cid=1922896'},
 {'_id': ObjectId('683dce7105923cb54314fdc2'),
  'URL': 'https://www.agoda.com/vi-vn/activities/detail?activityId=1107336&cityId=13170&currency=VND&cid=1922896'},
 {'_id': ObjectId('683dce7105923cb54314fdc3'),
  'URL': 'https://www.agoda.com/vi-vn/activities/detail?activityId=1137710&cityId=13170&currency=VND&cid=1922896'},
 {'_id': ObjectId('683d

In [14]:
len (items_to_scrape)

39093

In [23]:
driver = Create_WebDriver ()
idx = 1062
for item in items_to_scrape[1062:]:
    object_id = ObjectId(item['_id'])  # Lấy ID dưới dạng string
    url = item['URL']             # Lấy URL
    
    driver.get (url)
    time.sleep (5)
    
    print (f'🔄 Tiến hành scrape url thứ {idx} Object Id {object_id}')
    Scraping_Pages (driver, url, object_id)
    idx += 1
    
    # ⏳ Nghỉ lâu hơn sau mỗi 20 URL
    if idx % 50 == 0:
        print(f'😴 Nghỉ dài sau {idx} URL đã scrape...')
        random_delay(1, 2)  # Delay dài hơn
driver.quit ()

🔄 Tiến hành scrape url thứ 1062 Object Id 683dce7105923cb5431501e4
🔍 Đang scrape reviews trang thứ 1...
✅ Đã đến trang cuối cùng (nút Next bị vô hiệu hóa).
🔄 Tiến hành scrape url thứ 1063 Object Id 683dce7105923cb5431501e5
⏳ Không tìm thấy nội dung, thử lại (1/3)...
⏳ Không tìm thấy nội dung, thử lại (2/3)...
⏳ Không tìm thấy nội dung, thử lại (3/3)...
⚠️ Không thể tải nội dung sau 3 lần thử. Bỏ qua trang này.
🔄 Tiến hành scrape url thứ 1064 Object Id 683dce7105923cb5431501e6
⏳ Không tìm thấy nội dung, thử lại (1/3)...
⏳ Không tìm thấy nội dung, thử lại (2/3)...
⏳ Không tìm thấy nội dung, thử lại (3/3)...
⚠️ Không thể tải nội dung sau 3 lần thử. Bỏ qua trang này.
🔄 Tiến hành scrape url thứ 1065 Object Id 683dce7105923cb5431501e7
⏳ Không tìm thấy nội dung, thử lại (1/3)...
⏳ Không tìm thấy nội dung, thử lại (2/3)...
⏳ Không tìm thấy nội dung, thử lại (3/3)...
⚠️ Không thể tải nội dung sau 3 lần thử. Bỏ qua trang này.
🔄 Tiến hành scrape url thứ 1066 Object Id 683dce7105923cb5431501e8
🔍 Đ

MaxRetryError: HTTPConnectionPool(host='localhost', port=57800): Max retries exceeded with url: /session/051f8af53d96f7813964829788d1f05a/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12c166ad0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [12]:
driver.quit ()

In [ ]:
# import time
# def f():
#     for i in range(10):
#         time.sleep(1)
#         yield i